In [1]:
import pandas as pd
from catboost import CatBoostClassifier, Pool
import numpy as np
import gc

In [2]:
dtypes = {
        'app'           : 'uint16',
        'device'        : 'uint16',
        'os'            : 'uint16',
        'channel'       : 'uint16',
        'hour'          : 'uint8',
        'is_attributed' : 'uint8', 
        'ip_day_hour_count': 'uint32', 
        'ip_os_day_hour_count': 'uint32', 
        'ip_app_day_hour_count': 'uint32', 
        'ip_app_os_day_hour_count': 'uint32', 
        'app_day_hour_count': 'uint32', 
        'ip_device_os_count': 'uint32', 
        'ip_app_device_os_count': 'uint32', 
        'ip_device_os_mean': 'float16',
        'ip_app_device_os_mean': 'float16',
        'ip_app_device_mean': 'float16',
        'app_device_os_mean': 'float16',
        'ip_device_os_time2nextclick': 'int32',
        'ip_app_device_os_time2nextclick': 'int32',
        'ip_app_device_time2nextclick': 'int32',
        'ip_device_os_time2previousclick': 'int32',
        'ip_app_device_os_time2previousclick': 'int32',
        'ip_app_device_time2previousclick': 'int32',
        'ip_device_os_countfromfuture': 'uint32', 
        'ip_app_device_os_countfromfuture': 'uint32', 
        'ip_app_device_countfromfuture': 'uint32', 
        'ip_device_os_countfrompast': 'uint32', 
        'ip_app_device_os_countfrompast': 'uint32', 
        'ip_app_device_countfrompast': 'uint32', 
        'ip_device_os_lasttimediff': 'int32',
        'ip_app_device_os_lasttimediff': 'int32',
        'ip_app_device_lasttimediff': 'int32',
        'ip_device_os_firsttimediff': 'int32',
        'ip_app_device_os_firsttimediff': 'int32',
        'ip_app_device_firsttimediff': 'int32',
        'matrixFact_user_iposdeviceapp_item_app': 'float16',
        'matrixFact_user_ip_item_appdeviceos': 'float16',
        'matrixFact_user_ipchannel_item_appdeviceos': 'float16',
        'ip_device_os_regression': 'float16',
        'ip_app_device_os_regression': 'float16',
        'ip_app_device_regression': 'float16',
        'ip_app_device_os_channel_regression': 'float16', 
        'attributed_timediffmax':'int32',
        'attributed_timediffmin':'int32',
        'attributed_timediff':'float16',
        'matrixFact_user_ipappdeviceos_item_channel': 'float16'
    
        } 

In [3]:
# load_path = '/home/kai/data/kaggle/talkingdata/wl/data/equalhour/'
# file_format = '{}_equalhour_supplementV1.csv'
# day_list = ['day7', 'day8', 'day9']
# df_dict = {}
# for file in ['day7', 'day8', 'day9','test']: 
#     df_dict[file] = pd.read_csv(load_path+file_format.format(file))
#     print(file_format.format(file))
    

In [4]:
load_path = '/home/kai/data/kaggle/talkingdata/wl/data/equalhour/'
file_format = '{}_equalhour_supplementV1.csv'
loads = pd.read_csv(load_path+file_format.format('day9'), dtype=dtypes)

In [5]:
# loads = pd.concat([trainset, valset])

In [6]:
cut = int(0.8 * len(loads))
# trainset = loads.iloc[:cut]
# valset = loads.iloc[cut:]
trainset = loads.iloc[:100000]
valset = loads.iloc[100000:150000]
del loads
gc.collect()

23

# get feature cols

In [7]:
extra = set(['ip_device_os_countself',
             'ip_day_hour_countself', 
             'app_day_hour_countself',
             'ip_app_device_os_countself',
             'ip_app_day_hour_countself', 
             'ip_os_day_hour_countself', 
             'ip_app_os_day_hour_countself' ])

feature_cols = list(set(trainset.columns) - set(['is_attributed']) - extra)
categorical_col = [ 'app', 'device', 'os', 'channel', 'hour']
category_index = [feature_cols.index(cat) for cat in categorical_col]
feature_cols
target = 'is_attributed'

train_pool = Pool(trainset[feature_cols], trainset[target], cat_features=category_index)
val_pool = Pool(valset[feature_cols], valset[target], cat_features=category_index)

In [6]:
import sys
print(sys.getsizeof(train_pool))

80


In [1]:
import sys
print(sys.getsizeof(trainset))

NameError: name 'trainset' is not defined

# Train Catboost

In [12]:
from sklearn.metrics import roc_auc_score

import pdb 

class ROCMetric(object):
    def get_final_error(self, error, weight):
        return error

    def is_max_optimal(self):
        return True

    def evaluate(self, approxes, target, weight):
        # approxes - list of list-like objects (one object per approx dimension)
        # target - list-like object
        # weight - list-like object, can be None
        approx = approxes[0]
#         pdb.set_trace()
        roc = roc_auc_score(np.array(target), np.array(approx))
        return roc, 1
#         return roc, roc

In [13]:
param = {
        'eval_metric': ROCMetric(), 
#         'eval_metric': 'AUC', 
         'learning_rate':0.1, 
         'od_type':'Iter',
         'od_wait':50,
         'loss_function':'Logloss', 
         'depth':2, 
         'thread_count':20, 
         'iterations':5000,
         'scale_pos_weight': 398,
        'save_snapshot': True,
        'snapshot_file':'drft0403'
        }

model = CatBoostClassifier(**param)

In [14]:
model.fit(train_pool,  eval_set=val_pool, use_best_model=True, verbose_eval=1)

Can't load progress from file: catboost_info/drft0403 exception: (TCatboostException) catboost/libs/algo/learn_context.cpp:252: Saved model's Params are different from current model's params
0:	learn: 0.9249591	test: 0.9155487	best: 0.9155487 (0)	total: 94.4ms	remaining: 7m 51s
1:	learn: 0.9458564	test: 0.9331851	best: 0.9331851 (1)	total: 187ms	remaining: 7m 48s
2:	learn: 0.9508032	test: 0.9445310	best: 0.9445310 (2)	total: 267ms	remaining: 7m 24s
3:	learn: 0.9496083	test: 0.9467300	best: 0.9467300 (3)	total: 353ms	remaining: 7m 21s
4:	learn: 0.9503636	test: 0.9472131	best: 0.9472131 (4)	total: 434ms	remaining: 7m 13s
5:	learn: 0.9498691	test: 0.9470385	best: 0.9472131 (4)	total: 526ms	remaining: 7m 18s
6:	learn: 0.9502310	test: 0.9485360	best: 0.9485360 (6)	total: 610ms	remaining: 7m 15s
7:	learn: 0.9554812	test: 0.9471749	best: 0.9485360 (6)	total: 694ms	remaining: 7m 13s
8:	learn: 0.9558332	test: 0.9469974	best: 0.9485360 (6)	total: 779ms	remaining: 7m 11s
9:	learn: 0.9575013	test:

91:	learn: 0.9866466	test: 0.9616669	best: 0.9620557 (85)	total: 8.45s	remaining: 7m 30s
92:	learn: 0.9867041	test: 0.9616135	best: 0.9620557 (85)	total: 8.54s	remaining: 7m 30s
93:	learn: 0.9869032	test: 0.9615936	best: 0.9620557 (85)	total: 8.63s	remaining: 7m 30s
94:	learn: 0.9871400	test: 0.9618640	best: 0.9620557 (85)	total: 8.71s	remaining: 7m 29s
95:	learn: 0.9871359	test: 0.9616589	best: 0.9620557 (85)	total: 8.8s	remaining: 7m 29s
96:	learn: 0.9872146	test: 0.9615679	best: 0.9620557 (85)	total: 8.89s	remaining: 7m 29s
97:	learn: 0.9872716	test: 0.9614810	best: 0.9620557 (85)	total: 8.99s	remaining: 7m 29s
98:	learn: 0.9872262	test: 0.9613446	best: 0.9620557 (85)	total: 9.08s	remaining: 7m 29s
99:	learn: 0.9872984	test: 0.9616645	best: 0.9620557 (85)	total: 9.18s	remaining: 7m 29s
100:	learn: 0.9873302	test: 0.9616192	best: 0.9620557 (85)	total: 9.27s	remaining: 7m 29s
101:	learn: 0.9873529	test: 0.9615951	best: 0.9620557 (85)	total: 9.36s	remaining: 7m 29s
102:	learn: 0.987496

183:	learn: 0.9927742	test: 0.9647866	best: 0.9658206 (142)	total: 17.1s	remaining: 7m 27s
184:	learn: 0.9927767	test: 0.9648742	best: 0.9658206 (142)	total: 17.2s	remaining: 7m 27s
185:	learn: 0.9928171	test: 0.9648458	best: 0.9658206 (142)	total: 17.3s	remaining: 7m 27s
186:	learn: 0.9928293	test: 0.9648739	best: 0.9658206 (142)	total: 17.4s	remaining: 7m 27s
187:	learn: 0.9928652	test: 0.9650351	best: 0.9658206 (142)	total: 17.5s	remaining: 7m 26s
188:	learn: 0.9928785	test: 0.9649963	best: 0.9658206 (142)	total: 17.6s	remaining: 7m 26s
189:	learn: 0.9929515	test: 0.9654723	best: 0.9658206 (142)	total: 17.7s	remaining: 7m 26s
190:	learn: 0.9929537	test: 0.9655259	best: 0.9658206 (142)	total: 17.7s	remaining: 7m 26s
191:	learn: 0.9930065	test: 0.9647476	best: 0.9658206 (142)	total: 17.8s	remaining: 7m 26s
192:	learn: 0.9930710	test: 0.9649430	best: 0.9658206 (142)	total: 17.9s	remaining: 7m 26s
193:	learn: 0.9930780	test: 0.9647645	best: 0.9658206 (142)	total: 18s	remaining: 7m 26s
S

In [86]:
model.save_model('/home/kai/data/kaggle/talkingdata/wl/data/equalhour/catboost/draft')

In [89]:
aaa = CatBoostClassifier().load_model(fname='/home/kai/data/kaggle/talkingdata/wl/data/equalhour/catboost/draft')

In [91]:
bbb = aaa.predict_proba(valset[feature_cols], ntree_start=0, ntree_end=194)[:,1]

In [84]:
aaa = model.predict_proba(valset[feature_cols], ntree_start=0, ntree_end=194)[:,1]
aaa

array([ 0.0427769 ,  0.00872796,  0.00385234, ...,  0.01451301,
        0.00800453,  0.00869701])

In [92]:
from sklearn.metrics import roc_auc_score
print(roc_auc_score(valset[target].values, bbb))

0.968119030217


In [49]:
len(b)

194

In [52]:
a

array([[ 0.92542857,  0.07457143],
       [ 0.88862555,  0.11137445],
       [ 0.84014467,  0.15985533],
       ..., 
       [ 0.92542857,  0.07457143],
       [ 0.92542857,  0.07457143],
       [ 0.88330049,  0.11669951]])

In [108]:
from sklearn.metrics import roc_auc_score
print(roc_auc_score(valset[target].values, preds))

0.961529554947
